## **Pipelines & Transformers**

En esta notebook vamos a jugar con la creación de Pipelines y Transformers/Vectorizers para simplificar el desarrollo de un pipeline de procesamiento, que incluya desde el pre-procesamiento de los datos, hasta el entrenamiento y testing de un clasificador.


Como venimos trabajando, en la mayoría de las aplicaciones de Machine Learning, los datos con los que hay que trabajar no se encuentran en las condiciones óptimas para entrenar el "mejor" modelo posible.

Existen diversos pasos a realizar a las features, dependientes su tipo. Por ejemplo, puede haber un encoding para las features nominales o categóricas, escalado y normalización para features numéricas, y otras tantas alternativas de pre-procesamiento para el texto. Recordemos que los atributos textuales no son bien recibidos por los modelos a entrenar dado que los mismos esperan una representación numérica.

En las notebooks anteriores el procesamiento lo realizamos de forma "manual", es decir, creando algunos métodos para aplicar el pre-procesamiento a los datos de forma independiente de otras estructuras o tareas. Sin embargo, esta sepración e independencia puede resultar "tediosa". Si el objetivo último es entrenar un modelo, debemos recordar que hay que aplicar el pre-procesamiento tanto al training como al test, y luego a cada uno de los elementos que querramos evaluar con el modelo. Por otra parte, si quisieramos compartir nuestro modelo con otros, también tenemos que recordar compartir el pre-procesamiento de los datos aparte.

En este contexto, ``Scikit-learn`` nos provee de algunos mecanismos para simplificar la integración de las tareas y el proceso: los pipelines y los transformers, que permiten:

* Hacer que el workflow sea más fácil de leer y entender.
* Mejorar la "organización" del workflow.
* Incrementar la reproducibilidad del proceso.

#### Algunas definiciones

##### ``fit`` vs ``transform``

* ``fit`` encuentra los parámetros del modelo que será luego utilizado para transformar los datos. No necesariamente tiene que hacer algo más que retornar el objeto ``Transformer``, es decir, si mismo. 

* ``transform`` aplica la transformación a los datos de entrada, retornando los datos transformados.

* ``fit_transform`` aplica el ``fit`` y ``transform`` de forma consecutiva.

Cuando estamos usando un ``Pipeline``, ``fit`` y ``fit_transform`` tienen el mismo comportamiento salvo para el último elemento del pipeline. Para aquellos elementos previos al último realizan el ``fit`` de cada uno de los elementos del pipe y luego sus correspondientes ``transform``, mientras que para el último, el ``fit`` invoca al ``fit`` y el ``fit_transform`` al ``fit_transform``. 

En el caso de los estimadores, también tendremos el ``predict`` que aplica los ``transform`` sobre los datos y luego realiza el ``predict`` sobre el último elemento del pipeline.

Para este ejemplo, vamos a utilizar un dataset simple de tweets para la detección de *hate speech*, del que nos vamos a quedar con un atributo de tipo texto y la clase numérica (0: No es hate speech, 1: hate speech, 2: offensive speech).

In [ ]:
# Cargamos los datos necesarios
import pandas as pd

url = "https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv"
df = pd.read_csv(url, usecols=['class', 'tweet']) # de todas las columnas que tiene el dataset, nos vamos a quedar solo con el texto y la clase
df = df[60:65] # limitamos la cantidad de filas del dataframe

print(df)

In [ ]:
df[df['class']==1]

In [ ]:
import nltk
from collections import Counter

def posStats (textSample):
    # Función para contar las parts of speech
    # https://stackoverflow.com/questions/10674832/count-verbs-nouns-and-other-parts-of-speech-with-pythons-nltk
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    # print(str(textSample))
    tokens = nltk.word_tokenize(str(textSample))
    print(len(tokens))
    # print(tokens)
    text = nltk.Text(tokens)
    # print(len(text))
    tags = nltk.pos_tag(text)
    # print(len(tags))

    counts = Counter(tag for word,tag in tags)
    return counts

In [ ]:
print(posStats(df['tweet']))
print(posStats(df['tweet'].str.cat(sep=' ')))

Como el objetivo último de este transformer es combinarlo con un clasificador, vamos a crear un split de training y test para poder probar y ver el resultado de aplicar el ``Transformer`` y de integrarlo en el pipeline completo.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size = 0.20,random_state=42)

# recordemos que para entrenar tenemos separar la clase
X_train = train_set.drop("class", axis=1)  
y_train = train_set["class"].copy()

### Opción 1: Agregamos pre-procesamiento a los Transformers que ya existen en sklearn

Ya vienen incluidos algunos transformers para aplicar a atributos textuales. En este caso, vamos a tomar como base ``CountVectorizer``. Este vectorizer permite definir algunas configuraciones, como por ejemplo: (copiado de la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

* ``strip_accents : {‘ascii’, ‘unicode’}, default=None``.
Eliminar acentos y otras normalizaciones. ``ascii`` es más rápida, pero solo funciona con chars que tienen un mapping directo a ascii. ``unicode`` un poco más lento, pero funciona sobre todos los chars. Utilizan la normalización NFKD.

* ``lowercase bool, default=True``
Convierte todos los chars a minúscula antes de tokenizar.

* ``analyzer : {‘word’, ‘char’, ‘char_wb’} or callable, default=’word’``
Si se debe analizar a nivel palabra o a nivel n-chars. ``char_wb`` saca los n-chars de adentro de las palabras y lo que "sobre" es completado con espacios en blanco.

* ``stop_words : {‘english’}, list, default=None``
Solo se utiliza a nivel ``word`` de análisis. Se le puede pasar una lista de stopwords. 

* ``ngram_range : tuple (min_n, max_n), default=(1, 1)``
Tamaño de los n-grams a seleccionar. Se selecciona ``min_n <= n <= max_n``, por defecto se seleccionan palabras individuales (1-gram).

* ``min_df : float or int, default=1``
Setear la mínima frecuencia de documentos de una característica para ser incluido. Si se utiliza un float en el rango ``[0.0, 1.0]`` se indica la proporción de docuemntos.

* ``max_df : float or int, default=1.0``
Ignorar las característica que tienen una frecuencia a la definida. Si se utiliza un float en el rango ``[0.0, 1.0]`` se indica la proporción de docuemntos. De acuerdo con la documentación se puede utilizar en reemplazo de los stopwords si se setea en el rango ``[0.7, 1.0)``.

* ``max_features : int, default=None``
Solo incluir las características en el top-N ordenadas por su frecuencia.

* ``preprocessor : callable, default=None``
Sobre-escribir el pre-procesamiento pero mantener el tokenizer y la generación de n-grams.

* ``tokenizer : callable, default=None``
Sobre-escribir el paso de tokenización pero mantener el pre-procesamiento y la generación de n-grams.

Como se puede ver, la mayoría de los pasos que vimos de pre-procesamiento pueden ser configurados en el ``Vectorizer``. Sin embargo, las posibilidades están restringidas a las implementaciones de las mencionadas basadas en NLTK. Cualquier otra cosa por fuera de NLTK debemos implementarla nosotros, ya sea como parte del ``Vectorizer`` o previa a la transformación. Por ejemplo, agregar corrector ortográfico. 

En esta notebook vamos a incorporar el procesamiento a los ``Vectorizers``.

Si miramos la descripción de los diferentes parámetros que acepta el ``Vectorizer``, vamos a ver que ``preprocesor`` acepta un ``callable``, lo que significa que le podemos pasar un método para que sea ejecutado. Lo importante a considerar es que el método se ejecutará sobre cada **token**, no sobre el texto completo que reciba.

Vamos a implementar un método que aplique corrección ortográfica a los tokens que encuentre. Ese método lo vamos a usar como parámetro del ``preprocessor``.

In [ ]:
!pip install textblob

In [ ]:
from textblob import TextBlob
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('stopwords')

In [ ]:
def preprocess(s): 
  return str(TextBlob(s).correct()).lower()

count_transformer = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'),preprocessor=preprocess)

In [ ]:
transformed_tweets = count_transformer.fit_transform(df["tweet"])

Depende de la definición de los stopwords, puede generarse algún conflicto por el orden en el que se ejecutan los procesamientos. Fijense que si sacamos el ``lower()`` en nuestro procesamiento, los tokens no son convertidos a ``lowercase`` aun cuando el valor por default para ``lowercase`` es ``True``.

Veamos la diferencia del conjunto de características que obtenemos si no aplicamos nuestro ``preprocessor``.

In [ ]:
print(len(count_transformer.get_feature_names()))
print(sorted(count_transformer.get_feature_names()))

In [ ]:
count_normal = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'))

transformed_tweets_normal = count_normal.fit_transform(df["tweet"])

In [ ]:
print(len(count_normal.get_feature_names()))
print(sorted(count_normal.get_feature_names()))

Ahora, vamos a probar de usar nuestro ``Vectorizer`` en un pipeline completo. Para eso, vamos a definir un ``ColumnTransformer``, al que le vamos a agregar nuestro transformer, indicando la columna sobre la cual aplicarlo. Luego, agregamos al pipeline también el modelo elegido para entrenar y entrenamos!

Esto nos permite combinar el pre-procesamiento con el entrenamiento del modelo y de forma similar también vamos a poder hacer la evaluación del modelo. Notar que al incorporar el procesamiento al pipeline, solo tenemos que darle la estructura original que soporta nuestros tweets.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('count', count_transformer, "tweet")]) # importante definir las columnas sobre las cuales se aplica

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

rf.fit(X_train,y_train)      

En este pipeline implementamos un método para el ``preprocessor``, ahora vamos a modificar el ``tokenizer``. La diferencia con el anterior, es que se aplica sobre **todo el texto**.

Para este ejemplo, vamos a crear un tokenizer que nos reemplace los tokens que aparecen en el texto por su etiqueta POS, basándonos en spaCy.

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

def tokenizer(sent):
  toks = []
  for token in nlp(sent):
    toks.append(token.tag_)
  return toks

count_transformer_tokenizer = CountVectorizer(tokenizer=tokenizer)

In [ ]:
transformed_tweets_tokenizer = count_transformer_tokenizer.fit_transform(df["tweet"])

In [ ]:
print(len(count_transformer_tokenizer.get_feature_names()))
print(sorted(count_transformer_tokenizer.get_feature_names()))

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('count', count_transformer_tokenizer, "tweet")]) # importante definir las columnas sobre las cuales se aplica

rf = Pipeline(steps=[('preprocesor_tokenizer', preprocessor),
                      ('classifier', LogisticRegression())])   

rf.fit(X_train,y_train)   

Nota. El ``tokenizer`` y el ``preprocessor`` pueden combinarse.

### Opción 2: Creamos nuestro Transformer de cero

En la opción anterior agregamos nuestro procesamiento al ``Vectorizer`` que ya existe. En esta, vamos a crear nuestro ``Vectorizer`` de cero. Este caso nos permite agregar más procesamiento sin estar atados al agregado de comportamiento al ``tokenizer`` o al ``preprocessor``.

Si bien podemos agregar comportamiento más complejo, vamos a ejemplificar con el mismo comportamiento que le dimos al ``tokenizer``.

Para esto, debemos crear una nueva clase que extienda los ``Vectorizers`` para el comportamiento que nosotros queremos. En este sentido, tenemos que implementar algunos métodos:
* ``fit``. Preparar el modelo interno de nuestro ``Transformer``. Puede que no necesiten hacer nada. En este caso, vamos a hacer un ``fit`` de nuestro ``Vectorizer`` interno.
* ``inverse_transform``. Cuál es el resultado de deshacer la transformación? En este caso, no va a ser posible, con lo que retornamos una lista vacía.
* ``tranform``. El procesamiento propiamente dicho que queremos aplicar. 

La complicación que tiene esta implementación es el tipo de datos que se esperan que se retornen. En principio, la salida de esto va directo al modelo a entrenar, por lo que tenemos que retornar aquello que espera dicho ``Transformer``: una representación matricial de nuestros datos.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
class PartOfSpeech(BaseEstimator, TransformerMixin): # tweet transformado en su part of speech
  def __init__(self, stopwords = None, punct=None, lower=True, strip=True):
    self.lower = lower
    self.strip      = strip
    self.stopwords  = stopwords 
    self.punct      = r'[!?.,()\":$]'
    self.nlp = spacy.load('en_core_web_sm')
    self.counter = CountVectorizer()

  def fit(self,X,y=None):
    self.counter.fit([' '.join(self.process(str(doc))) for doc in X.values]) # hacemos el fit de nuestro vectorizer interno
    return self

  def inverse_transform(self, X):
    return []
  
  def transform(self,X): # acá tenemos que ser cuidadosos. Recordemos que la salida de esto en principio va directo al modelo a entrenar, es por eso que tenemos que retornar
    return self.counter.transform(raw_documents=[' '.join(self.process(str(doc))) for doc in X.values])

  def process(self,doc):
    proc = []
    doc = re.sub(self.punct, '', doc)
    for token in self.nlp(doc):
      proc.append(token.tag_)
    return proc

Finalmente, vamos a ver que se puede utilizar:

In [ ]:
pos_processing = PartOfSpeech()

preprocessor = ColumnTransformer(
    transformers=[
        ('pos', pos_processing, "tweet")]) # importante definir las columnas sobre las cuales se aplica

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

rf.fit(X_train,y_train) 

Vamos a realizar la predicción.

In [ ]:
print(rf.predict(test_set.drop('class',axis=1)))

print(test_set['class'])